# 지연 속성에는 `__getattr__, __getattribute__, __setattr__`을 사용하자

Python의 언어 후크를 이용하면 시스템들을 연계하는 범용 코드를 쉽게 만들 수 있다. 예를 들어 데이터베이스의 row를 파이썬 객체로 표현한다고 하자. 데이터베이스에는 스키마 세트가 있다. 그러므로 row에 대응하는 객체를 사용하는 코드는 데이터베이스 형태도 알아야 한다. 하지만 Python에서는 객체와 데이터베이스를 연결하는 코드에서 row의 스키마를 몰라도 된다. 코드를 범용으로 만들면 된다.

사용하기에 앞서 정의부터 해야 하는 일반 인스턴스 속성, `@property` 메서드, 디스크립터로는 이렇게 할 수 없다. Python은 `__getattr__`이라는 특별한 메서드로 이런 동적 동작을 가능하게 한다. 클래스에 `__getattr__` 메서드를 정의하면 객체의 인스턴스 딕셔너리에 속성을 찾을 수 없을 때마다 이 메서드가 호출된다.

In [7]:
class LazyDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattr__(self, name):
        value = 'Value for %s' % name
        setattr(self, name, value)
        return value

이제 존재하지 않는 속성인 foo에 접근해보자. 그러면 파이썬이 `__getattr__`메서드를 호출하게 되고 이어서 인스턴스 딕셔너리 `__dict__`를 변경하게 된다.

In [8]:
data = LazyDB()
print('Before:', data.__dict__)
print('foo:  ', data.foo)
print('After: ', data.__dict__)

Before: {'exists': 5}
foo:   Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


`__getattr__`이 호출되는 시점을 보여주기 위해 LazyDB에 로깅 추가

무한 루프를 피하려고 `super().__getattr__()`로 실제 property 값을 얻어오는 부분을 눈여겨보자.

In [9]:
class LoggingLazyDB(LazyDB):
    def __getattr__(self, name):
        print('Called __getattr__(%s)' % name)
        return super().__getattr__(name)
    
data = LoggingLazyDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)
print('After: ', data.__dict__)

exists: 5
Called __getattr__(foo)
foo:    Value for foo
foo:    Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


exists 속성은 인스턴스 딕셔너리에 있으므로 `__getattr__`이 절대 호출되지 않는다. foo 속성은 원래는 인스턴스 딕셔너리에 없으므로 처음에는 `__getattr__`이 호출된다. 하지만 foo에 대응하는 `__getattr__`호출은 setattr을 호출하며, setattr은 인스턴스 dictioinary에 foo를 저장한다. 따라서 foo에 두 번째로 접근할 때는 `__getattr__`이 호출되지 않는다.

이런 동작은 schemaless data에 지연 접근하는 경우에 특히 도움이 된다. `__getattr__`이 property loading이라는 어려운 작업을 한 번만 실행하면 다음 접근부터는 기존 결과를 가져온다.

`__getattr__` 후크는 기존 속성에 빠르게 접근하려고 객체의 인스턴스 딕셔너리를 사용할 것이므로 이 작업에는 믿고 쓸 수 없다.

Python에는 쓰임새를 고려한 `__getattribute__`라는 또 다른 후크가 있다. 이 특별한 메서드는 객체의 속성에 접근할 때마다 호출되며, 심지어 해당 속성이 속성 딕셔너리에 있을 때도 호출된다. 이런 동작 덕분에 속성에 접근할 때마다 전역 트랜잭션 상태를 확인하는 작업 등에 쓸 수 있다. 여기서는 `__getattribute__`가 호출될 때마다 로그를 남기려고 ValidatingDB를 정의한다.

In [10]:
class ValidatingDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value
        
data = ValidatingDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

Called __getattribute__(exists)
exists: 5
Called __getattribute__(foo)
foo:    Value for foo
Called __getattribute__(foo)
foo:    Value for foo


동적으로 접근한 property가 존재하지 않아야 하는 경우에는 AttributeError를 일으켜서 `__getattr__, __getattribute__`에 속성이 없는 경우의 Python 표준 동작이 일어나게 한다.

In [11]:
class MissingPropertyDB(object):
    def __getattr__(self, name):
        if name == 'bad_name':
            raise AttributeError('%s is missing' % name)
            
data = MissingPropertyDB()
data.bad_name

AttributeError: bad_name is missing

Python 코드로 범용적인 기능을 구현할 때 종종 내장 함수 hasattr로 property가 있는지 확인하고 내장 함수 getattr로 property 값을 가져온다. 이 함수들도 `__getattr__`을 호출하기 전에 instance dictionary에서 속성 이름을 찾는다.

In [12]:
data = LoggingLazyDB()
print('Before:     ', data.__dict__)
print('foo exists: ', hasattr(data, 'foo'))
print('After:      ', data.__dict__)
print('foo exists: ', hasattr(data, 'foo'))

Before:      {'exists': 5}
Called __getattr__(foo)
foo exists:  True
After:       {'exists': 5, 'foo': 'Value for foo'}
foo exists:  True


위 예제에서 `__getattr__`은 한 번만 호출된다. 이와 대조로, `__getattribute__`를 구현한 클래스인 경우 객체에 hasattr이나 getattr을 호출할 때마다 `__getattribute__`가 실행된다.

In [13]:
data = ValidatingDB()
print('foo exists: ', hasattr(data, 'foo'))
print('foo exists: ', hasattr(data, 'foo'))

Called __getattribute__(foo)
foo exists:  True
Called __getattribute__(foo)
foo exists:  True


이제 파이썬 객체에 값을 할당할 때 지연 방식으로 데이터를 데이터베이스에 집어넣고 싶다고 해보자. 이 작업은 임의 속성 할당을 가로채는 `__setattr__`언어 후크로 할 수 있다. `__getattr__`과 `__getattribute__`로 속성을 추출하는 것과는 다르게 별도의 메서드 두 개가 필요하지 않다. `__setattr__` 메서드는 인스턴스 속성이 할당 받을 때마다 직접 혹은 내장 함수 setattr을 통해 호출된다.

In [14]:
class SavingDB(object):
    def __setattr__(self, name, value):
        # 몇몇 데이터를 DB 로그로 저장함
        # ...
        super().__setattr__(name, value)

In [15]:
class LoggingSavingDB(SavingDB):
    def __setattr__(self, name, value):
        print('Called __setattr__(%s, %r)' % (name, value))
        super().__setattr__(name, value)

In [16]:
data = LoggingSavingDB()
print('Before: ', data.__dict__)
data.foo = 5
print('After:  ', data.__dict__)
data.foo = 7
print('Finally:', data.__dict__)

Before:  {}
Called __setattr__(foo, 5)
After:   {'foo': 5}
Called __setattr__(foo, 7)
Finally: {'foo': 7}


`__getattribute__`와 `__setattr__`을 사용할 때 부딪히는 문제는 객체의 속성에 접근할 때마다 호출된다는 점이다.

In [17]:
class BrokenDcitionaryDB(object):
    def __init__(self, data):
        self._data = {}
        
    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        return self._data[name]

그러려면 위와 같이 `__getattribute__` 메서드에서 `self._data`에 접근해야한다. 하지만 실제로 시도해보면 Python이 stack의 한계에 도달할 때까지 재귀 호출을 하게 되어 중단된다.

In [18]:
data = BrokenDcitionaryDB({'foo': 3})
data.foo

Called __getattribute__(foo)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __g

Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called _

RecursionError: maximum recursion depth exceeded while calling a Python object

문제는 `__getattribute__`가 `self._data`에 접근하면 `__getattribute__`가 다시 실행되고, 다시 `self._data`에 접근한다는 점이다. 해결책은 인스턴스에서 `super().__getattribute__` 메서드로 인스턴스 속성 딕셔너리에서 값을 얻어 오는 것이다.

In [19]:
class DictionaryDB(object):
    def __init__(self, data):
        self._data = data
        
    def __getattribute__(self, name):
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

In [ ]:
data = DictionaryDB()
print('Before')